# Plant Seedlings Classification

https://www.kaggle.com/c/plant-seedlings-classification/data

Determine the species of a seedling from an image

## Load Data

In [1]:
import tensorflow as tf
from tensorflow import keras
import pathlib
import numpy as np
import matplotlib.pyplot as plt
import os
import PIL.Image as Image
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [2]:
data_dir = pathlib.Path('/notebooks/storage/kaggle-solutions/Kaggle-Solutions/data/plant-seedlings-classification')
CLASS_NAMES = np.sort(np.array([item.name for item in data_dir.glob('train/*')]), axis=0)
image_count = len(list(data_dir.glob('train/*/*.png')))
CLASS_NAMES

array(['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed',
       'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize',
       'Scentless Mayweed', 'Shepherds Purse',
       'Small-flowered Cranesbill', 'Sugar beet'], dtype='<U25')

In [3]:
BATCH_SIZE = 32
IMG_HEIGHT = 224
IMG_WIDTH = 224
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)

In [4]:
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
train_dir = '/notebooks/storage/kaggle-solutions/Kaggle-Solutions/data/plant-seedlings-classification/train'
classes = ['Black-grass', 'Charlock', 'Cleavers', 'Common Chickweed',
       'Common wheat', 'Fat Hen', 'Loose Silky-bent', 'Maize',
       'Scentless Mayweed', 'Shepherds Purse',
       'Small-flowered Cranesbill', 'Sugar beet']
train_data_gen = image_generator.flow_from_directory(directory=train_dir,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = classes)

Found 4750 images belonging to 12 classes.


## Create Model

Very basic model

In [5]:
model_v1 = keras.Sequential()
model_v1.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model_v1.add(keras.layers.MaxPooling2D((2, 2)))
model_v1.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_v1.add(keras.layers.MaxPooling2D((2, 2)))
model_v1.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
model_v1.add(keras.layers.Flatten())
model_v1.add(keras.layers.Dense(64, activation='relu'))
model_v1.add(keras.layers.Dense(12, activation='softmax'))
model_v1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        36928     
_________________________________________________________________
flatten (Flatten)            (None, 173056)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                1

## Train Model

In [6]:
model_v1.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [7]:
model_v1.fit(train_data_gen, epochs=7, steps_per_epoch=None)

Train for 149 steps
Epoch 1/7
149/149 [==============================] - 74s 499ms/step - loss: 1.9537 - accuracy: 0.3436
Epoch 2/7
149/149 [==============================] - 70s 468ms/step - loss: 1.2506 - accuracy: 0.5718
Epoch 3/7
149/149 [==============================] - 71s 477ms/step - loss: 1.0057 - accuracy: 0.6659
Epoch 4/7
149/149 [==============================] - 69s 460ms/step - loss: 0.8134 - accuracy: 0.7333
Epoch 5/7
149/149 [==============================] - 69s 462ms/step - loss: 0.6213 - accuracy: 0.8019
Epoch 6/7
149/149 [==============================] - 69s 461ms/step - loss: 0.4433 - accuracy: 0.8562
Epoch 7/7
149/149 [==============================] - 69s 461ms/step - loss: 0.3339 - accuracy: 0.8941


## Evaluate on Test Data

In [31]:
test_dir = '/notebooks/storage/kaggle-solutions/Kaggle-Solutions/data/plant-seedlings-classification/'
image_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255)
test_data_gen = image_generator.flow_from_directory(directory=test_dir,
                                                     batch_size=BATCH_SIZE,
                                                     shuffle=True,
                                                     target_size=(IMG_HEIGHT, IMG_WIDTH),
                                                     classes = ['test'])

Found 794 images belonging to 1 classes.


In [36]:
pred = model_v1.predict(
    test_data_gen, batch_size=None, verbose=0, steps=None, callbacks=None, max_queue_size=10,
    workers=1, use_multiprocessing=False,
)
len(pred)

794

In [37]:
y = np.argmax(pred, axis=1)
len(y)

794

In [38]:
predicted_classes = [classes[i] for i in y]
files = [path[5:] for path in test_data_gen.filenames]
predicted_classes
len(files)
len(predicted_classes)

794

In [39]:
subm = np.stack([files,predicted_classes], axis=1)
np.savetxt('submission.csv', subm, fmt='%s,%s', header='file,species', comments='')